# Prostate Cancer Worshop

## Initial analysis

### Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew, kurtosis
from IPython.display import display

from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from feature_engine.outliers import Winsorizer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from bayes_opt import BayesianOptimization
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
from xgboost import XGBClassifier


### LoadingData

In [ ]:
df_train = pq.read_table('data/df_train.parquet').to_pandas()
df_test = pq.read_table('data/df_test.parquet').to_pandas()

df_train.shape

In [ ]:
df_train.head()

In [4]:
numeric_columns = [
    'Cant_gr_flia', 
    'Cant_riesgos_flia_mean', 
    'cantidad_serv_flia', 
    'CANTIDAD_SERVICIOS', 
    'conteo_dx_diferentes', 
    'EDAD', 
    'psa_max_gr_flia', 
    'psa_min_gr_flia', 
    'Pendiente', 
    'Pendiente_flia', 
    'Promedio_costo', 
    'Promedio_costo_flia', 
    'psa_max_gr_flia', 
    'psa_min_gr_flia', 
    'MEDICAMENTOS', 
    'MEDICINA ESPECIALIZADA', 
    'MEDICINA GENERAL', 
    'TIEMPO_AFILIACION', 
    'TIEMPO_ULTIMA_CITA', 
    'PERDIDA_DE_PESO', 
    'Intercepto', 
    'Intercepto_flia', 
    'Target',
    'Cant_Fliar_CP', 
    'Cant_Fliar_riesgos'
]

categorical_columns = [
    'AGRUPACION_DIASTOLICA', 
    'AGRUPACION_SISTOLICA', 
    'CANCER_MAMA_FAMILIAR', 
    'CANCER_OTRO_SITIO', 
    'CORONARIOS', 
    'CANCER_OTRO_SITIO_FAMILIAR',
    'CORONARIOS_FAMILIAR', 
    'CEREBRAL', 
    'CEREBRAL_FAMILIAR', 
    'DIABETES', 
    'DIABETES_FAMILIAR', 
    'ENFERMEDAD_RENAL', 
    'ENFERMEDAD_RENAL_FAMILIAR', 
    'HIPERTENSION', 
    'HIPERTENSION_FAMILIAR', 
    'OTROS_ANTECEDENTES_VASCULARES', 
    'RIESGOS', 
    'ESTADO_CIVI', 
    'estrato', 
    'parentesco', 
    'PROGRAMA', 
]

In [5]:
ordinal_columns = [
    'AGRUPACION_DIASTOLICA',
    'AGRUPACION_SISTOLICA',
    'HIPERTENSION',
    'HIPERTENSION_FAMILIAR',
    'RIESGOS',
    'estrato'
]

nominal_columns = [
    'CANCER_MAMA_FAMILIAR',
    'CANCER_OTRO_SITIO',
    'CORONARIOS',
    'CANCER_OTRO_SITIO_FAMILIAR',
    'CORONARIOS_FAMILIAR',
    'CEREBRAL',
    'CEREBRAL_FAMILIAR',
    'DIABETES',
    'DIABETES_FAMILIAR',
    'ENFERMEDAD_RENAL',
    'ENFERMEDAD_RENAL_FAMILIAR',
    'OTROS_ANTECEDENTES_VASCULARES',
    'ESTADO_CIVI',
    'parentesco',
    'PROGRAMA'                  
]



### Feature Importance

In [ ]:
df_encoded = df_train.copy()
for column in ordinal_columns + nominal_columns + ['IMC']:
    df_encoded[column] = df_encoded[column].astype('category')
X = df_encoded.drop(columns=['Target'])
y = df_encoded['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', enable_categorical=True)
model.fit(X_train, y_train)

importances = model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='teal')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Feature Importance using XGBoost')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [7]:
X_train_small = X_train.sample(frac=0.1, random_state=42)
y_train_small = y_train.loc[X_train_small.index] 

In [8]:
features_to_drop = ['Cant_Fliar_riesgos', 'Cant_Fliar_CP', 'min_Tiempo_CP_Fliar', 'psa_min_gr_flia', 'psa_max_gr_flia', 'CANCER_MAMA_FAMILIAR', 'Target']

### Validate dropping features
- In order to be sure whether we decide to drop or not the already identified features, we will run a preliminary model to test with and without the features

In [ ]:
def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', enable_categorical=True)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Model 1: With all features
X_all_features = df_encoded.drop(columns=['Target'])
y = df_encoded['Target']

f1_all_features = train_and_evaluate(X_all_features, y)
print(f"F1 Score with all features: {f1_all_features}")

# Model 2: Dropping variables with zero importance
X_reduced_features = df_encoded.drop(columns=['Target'] + features_to_drop)

f1_reduced_features = train_and_evaluate(X_reduced_features, y)
print(f"F1 Score after dropping zero-importance features: {f1_reduced_features}")

After dropping additional features, including `'Cant_Fliar_riesgos'`, `'Cant_Fliar_CP'`, `'min_Tiempo_CP_Fliar'`, `'psa_min_gr_flia'`, `'psa_max_gr_flia'`, and `'CANCER_MAMA_FAMILIAR'`, the model's performance improved. The F1 score increased from **0.5207** (with all features) to **0.5384** (after removing these features), indicating that simplifying the model by excluding both zero-importance features and those with minimal predictive power can enhance the model’s performance. By reducing noise from less significant features, the model was able to generalize better and make more accurate predictions, showcasing the benefits of feature selection in machine learning.

## Preprocessing Pipeline_________________________________________________

In [10]:
updated_numeric_columns = [col for col in numeric_columns if col not in features_to_drop]
updated_ordinal_columns = [col for col in ordinal_columns if col not in features_to_drop]
updated_nominal_columns = [col for col in nominal_columns if col not in features_to_drop]

In [11]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('winsorizer', Winsorizer(capping_method='quantiles', tail='right', fold=0.05)),
])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, updated_numeric_columns),
        ('ord', ordinal_transformer, updated_ordinal_columns),
        ('nom', nominal_transformer, updated_nominal_columns)
    ]
)

pipeline = Pipeline(steps=[
    ('drop_columns', 'passthrough'),
    ('preprocessor', preprocessor)
])

#### Applying preprocessor pipeline
- Imputation and dropping

In [12]:
X = df_train.drop(columns=features_to_drop)
y = df_train['Target']

pipeline.fit(X)
X_train_transformed = pipeline.transform(X)


#### Converting the pipeline output into a readable data frame

In [ ]:
transformed_columns = (
    updated_numeric_columns + 
    updated_ordinal_columns + 
    list(pipeline.named_steps['preprocessor'].transformers_[2][1]['onehot'].get_feature_names_out(updated_nominal_columns))
)

X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=transformed_columns)
X_train_transformed_df

<span style="color:red">Revisar el conteo de valores atipicos !!!!!!!!!!!!!</span>

In [14]:
def calculate_iqr(df, numeric_columns):
    """
    This function takes a dataframe and returns a dataframe that contains 
    the Interquartile Range (IQR) for each numeric column in the dataframe.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    
    Returns:
    pd.DataFrame: Dataframe containing IQR values for each numeric column
    """
    # Select numeric columns from the dataframe
    df_numeric_columns = df[numeric_columns]
    
    # Calculate Q1 (25th percentile) and Q3 (75th percentile) for each numeric column
    Q1 = df_numeric_columns.quantile(0.25)
    Q3 = df_numeric_columns.quantile(0.75)
    
    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
    
    # Create a dataframe to store the IQR values
    iqr_df = pd.DataFrame({
        'Column': IQR.index,
        'IQR': IQR.values
    }).sort_values(by='IQR', ascending=False)
    
    return iqr_df


In [ ]:
iqr_result = calculate_iqr(df_train, numeric_columns)
iqr_result

In [ ]:
iqr_result_after = calculate_iqr(X_train_transformed_df, updated_numeric_columns)
print(iqr_result_after)

### PCA .....

## Bayesian Optimization

In [17]:
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction


In [18]:
X_train = X_train_small
y_train = y_train_small

In [19]:
# Updated preprocessing pipeline
# Make sure you have the correct categorization of columns: numeric, ordinal, and nominal

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('winsorizer', Winsorizer(capping_method='quantiles', tail='right', fold=0.05)),
])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Added dense output for compatibility
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, updated_numeric_columns),  # Process numeric columns
        ('ord', ordinal_transformer, updated_ordinal_columns),  # Process ordinal columns
        ('nom', nominal_transformer, updated_nominal_columns)   # Process nominal columns
    ]
)

# Ensure that the preprocessing pipeline is used before PCA in your pipeline
# Number of components for PCA (adjust as needed)
n_components_pca = 5  # Adjust based on your dataset

# Define the SVM evaluation function with the updated pipeline
def svm_evaluate(C, gamma, kernel_choice):
    kernel = 'linear' if kernel_choice < 0.5 else 'rbf'
    
    # Create a complete pipeline: PCA + preprocessing + SVM
    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),  # Include the preprocessing pipeline
        ('pca', PCA(n_components=n_components_pca, random_state=42)),  # Add PCA after preprocessing
        ('svm', SVC(C=C, gamma=gamma, kernel=kernel, probability=True))  # SVM with hyperparameters
    ])
    
    # Perform K-fold cross-validation and return mean ROC AUC score
    roc_auc_scores = cross_val_score(model_pipeline, X_train, y_train, cv=3, scoring='roc_auc', verbose=0)
    
    return roc_auc_scores.mean()

# Define the parameter bounds for Bayesian Optimization
pbounds = {
    'C': (1, 5),       # Regularization parameter
    'gamma': (0.0001, 1),  # Kernel coefficient for 'rbf'
    'kernel_choice': (0, 1)  # 0 for 'linear', 1 for 'rbf'
}

# Set up the Bayesian optimizer
optimizer = BayesianOptimization(
    f=svm_evaluate,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Run the optimization without the progress bar
optimizer.maximize(init_points=2, n_iter=3)

# Output the best parameters
best_params = optimizer.max
print("Best parameters found:", best_params)

# Train the final SVM model with the best parameters
C_opt = best_params['params']['C']
gamma_opt = best_params['params']['gamma']
kernel_opt = 'linear' if best_params['params']['kernel_choice'] < 0.5 else 'rbf'

# Final pipeline with best hyperparameters
best_svm_model = Pipeline([
    ('preprocessor', preprocessor),  # Include the preprocessing pipeline
    ('pca', PCA(n_components=n_components_pca, random_state=42)),  # Add PCA
    ('svm', SVC(C=C_opt, gamma=gamma_opt, kernel=kernel_opt, probability=True))  # Best SVM model
])


In [ ]:
# Fit the final model
best_svm_model.fit(X_train, y_train)

# Evaluate the final model on the test set using ROC AUC
y_pred_proba = best_svm_model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class
test_roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Final ROC AUC Score on the test set: {test_roc_auc:.4f}")

Usar diferentes kernel

In [ ]:


# # Train the final SVM model with the best parameters
# C_opt = best_params['params']['C']
# gamma_opt = best_params['params']['gamma']
# kernel_opt = 'linear' if best_params['params']['kernel_choice'] < 0.5 else 'rbf'

# # Final pipeline with best hyperparameters
# best_svm_model = Pipeline([
#     ('preprocessor', preprocessor),  # Include the preprocessing pipeline
#     ('svm', SVC(C=C_opt, gamma=gamma_opt, kernel=kernel_opt, probability=True))
# ])

# # Fit the final model
# best_svm_model.fit(X_train, y_train)

# # Evaluate the final model on the test set using ROC AUC
# y_pred_proba = best_svm_model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class
# test_roc_auc = roc_auc_score(y_test, y_pred_proba)
# print(f"Final ROC AUC Score on the test set: {test_roc_auc:.4f}")
